<a href="https://colab.research.google.com/github/do2149/Wine-quality-indentification/blob/main/Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/paper')
!pwd

Mounted at /content/drive
/content/drive/My Drive/paper


In [ ]:
import pandas as pd
from numpy import *;#导入numpy的库函数
import numpy as np; #这个方式使用numpy的函数时，需要以np.开头。
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv("data.csv")      
feature = pd.read_excel("features.xlsx")
label = pd.read_excel("label.xlsx")
#feature.head()
#label.head()
data.head()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,3,11.6,0.580,0.66,2.20,0.074,10.0,47.0,1.00080,3.25,0.57,9.0
1,3,10.4,0.610,0.49,2.10,0.200,5.0,16.0,0.99940,3.16,0.63,8.4
2,3,7.4,1.185,0.00,4.25,0.097,5.0,14.0,0.99660,3.63,0.54,10.7
3,3,10.4,0.440,0.42,1.50,0.145,34.0,48.0,0.99832,3.38,0.86,9.9
4,3,8.3,1.020,0.02,3.40,0.084,6.0,11.0,0.99892,3.48,0.49,11.0


In [ ]:
import collections
print(collections.Counter(data.get('quality')))

Counter({5: 681, 6: 638, 7: 199, 4: 53, 8: 18, 3: 10})


# LES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression,LogisticRegression

def LES(Xtrain, ytrain, Xtest,ytest):
  model = LinearRegression()
 
  model.fit(Xtrain,ytrain)#线性回归训练
  
  a  = model.intercept_#截距
  b = model.coef_#回归系数
  print("拟合参数:截距",a,",回归系数：",b)
  
  #显示线性方程，并限制参数的小数位为两位
  print("最佳拟合线: Y = ",round(a,2),"+",round(b[0],2),"* X1 + ",round(b[1],2),"* X2")
  
  Y_pred = model.predict(Xtest)#对测试集数据，用predict函数预测
  a = []
  for item in Y_pred:
    if item <=0:
      a.append(-1)
    else:
      a.append(1)
  return a 

def LRS(Xtrain, ytrain, Xtest,ytest):
  model = LogisticRegression()
 
  model.fit(Xtrain,ytrain)#线性回归训练
  
  a  = model.intercept_#截距
  b = model.coef_#回归系数
  print("拟合参数:截距",a,",回归系数：",b)
  
  #显示线性方程，并限制参数的小数位为两位
  #print("最佳拟合线: Y = ",round(a,2),"+",round(b[0],2),"* X1 + ",round(b[1],2),"* X2")
  
  Y_pred = model.predict(Xtest)#对测试集数据，用predict函数预测
  
  return Y_pred  

过采样

In [ ]:
def upfirdn_pca(data_x, data_y):
  from imblearn.over_sampling import SMOTE, ADASYN
  X_resampled, y_resampled = SMOTE(kind='borderline1').fit_resample(data_x, data_y)
  #X_resampled, y_resampled = ADASYN(random_state=42).fit_resample(data_x, data_y)  
  #print(sorted(Counter(y_resampled).items())) 
  return X_resampled,y_resampled

random forest

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def RandomForest_para_select(X_train, y_train):
  score_lt = []

  # 每隔10步建立一个随机森林，获得不同n_estimators的得分
  for i in range(0,200,10):
      rfc = RandomForestClassifier(n_estimators=i+1
                                  ,random_state=90)
      score = cross_val_score(rfc, X_train, y_train, cv=8).mean()
      score_lt.append(score)
  score_max = max(score_lt)
  print('最大得分：{}'.format(score_max),        '子树数量为：{}'.format(score_lt.index(score_max)*10+1))
  n_e = score_lt.index(score_max)*10+1
  score_lt = []
  for i in range(n_e-11,n_e+9):
      rfc = RandomForestClassifier(n_estimators=i
                                  ,random_state=90)
      score = cross_val_score(rfc, X_train, y_train, cv=8).mean()
      score_lt.append(score)
  score_max = max(score_lt)
  #print('最大得分：{}'.format(score_max),        '子树数量为：{}'.format(score_lt.index(score_max)+30))
  # 绘制学习曲线
  rfc = RandomForestClassifier(n_estimators=score_lt.index(score_max)+30, random_state=90)

  # 用网格搜索调整max_depth
  param_grid = {'max_depth':np.arange(1,20)}
  GS = GridSearchCV(rfc, param_grid, cv=8)
  GS.fit(X_train, y_train)

  best_param = GS.best_params_
  best_score = GS.best_score_
  print(best_param, best_score)

  param_grid = {'max_features':np.arange(3,12)}
  rfc = RandomForestClassifier(n_estimators=score_lt.index(score_max)+30, random_state=90,max_depth=best_param.get('max_depth'))
  GS = GridSearchCV(rfc, param_grid, cv=8)
  GS.fit(X_train, y_train)
  best_param_1 = GS.best_params_
  best_score_1 = GS.best_score_
  print(best_param_1, best_score_1) 
  return score_lt.index(score_max)+30, best_param.get('max_depth'), best_param_1.get('max_features')

In [ ]:
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor  
import numpy as np  

def RandomForest(X_train, y_train, Xtest,ytest):
  #X_train,y_train = upfirdn_pca(X_train, y_train)
  rf2 = RandomForestRegressor()
  rf2.fit(X_train,y_train)
  pre_label = rf2.predict(Xtest)
  a = []
  for item in pre_label:
    if item <=0:
      a.append(-1)
    else:
      a.append(1)
  return a 
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor  
import numpy as np  

   

In [ ]:
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestClassifier  
import numpy as np  
from sklearn.model_selection import train_test_split

def RandomForest_2(X_train, y_train, Xtest,ytest):
  
  from warnings import simplefilter
  simplefilter(action='ignore', category=FutureWarning)
  rf2 = RandomForestClassifier(n_estimators=191,max_depth=18,max_features=3)
  rf2.fit(X_train,y_train)
  pre_label = rf2.predict(Xtest)
  
  return pre_label 

   

SVM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline
from collections import Counter
def SVM(X_train, y_train, Xtest,ytest):
  #X_train,y_train = upfirdn_pca(X_train, y_train)
  
  pca = PCA(n_components=11, whiten=True, random_state=42)
  #pca = KernelPCA(n_components=7,kernel="sigmoid",gamma = 0.0389, fit_inverse_transform=True)
  #Xtrain,ytrain = upfirdn_pca(Xtrain, ytrain)
  svc = SVC(kernel='poly',degree=5, class_weight='balanced',decision_function_shape='ovo',tol=0.00001)

  model = make_pipeline(pca, svc)
  param_grid = {'svc__C':[1,10,50,100]}#设置的C可能的值是1，2，5，10，可以自由设置
  grid = GridSearchCV(model,param_grid)
  %time grid.fit(X_train,y_train)
  model=grid.best_estimator_
  yfit=model.predict(Xtest)
  return yfit

KNN

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import EmpiricalCovariance, MinCovDet


def KNN(X_train, y_train, Xtest,ytest):
  #X_train,y_train = upfirdn_pca(X_train, y_train)
  label = data[['quality']]
  a = []
  for item in label['quality']:
    b = [item]
    a.append(b)
  arr = array(a)
  #X_train,y_train = upfirdn_pca(X_train, y_train)
  # fit a Minimum Covariance Determinant (MCD) robust estimator to data
  robust_cov = MinCovDet().fit(feature.values)
  # compare estimators learnt from the full data set with true parameters
  emp_cov = EmpiricalCovariance().fit(feature.values)
  robust_cov_y = MinCovDet().fit(arr)
  # compare estimators learnt from the full data set with true parameters
  emp_cov_y = EmpiricalCovariance().fit(arr)
  knn = KNeighborsClassifier(n_neighbors=11,
                           weights='distance',
                           algorithm='brute',
                           leaf_size=30,
                           metric='mahalanobis',
                           metric_params={'V': emp_cov},
                           n_jobs=1)
  
  #param_grid = {'svc__C':[1,2,5,10]}#设置的C可能的值是1，2，5，10，可以自由设置
  #grid = GridSearchCV(model,param_grid)
  
  #定义一个knn分类器对象
  knn.fit(X_train, y_train)
  y_predict = knn.predict(Xtest) 
  return y_predict
  

单个分类器测试

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(feature, label,test_size=0.2, stratify=label)
#Xtrain,ytrain = upfirdn_pca(Xtrain, ytrain)
#RandomForest_para_select(Xtrain, ytrain)

predict_label = SVM(Xtrain, ytrain.values.ravel(), Xtest,ytest)
#predict_label = KNN(Xtrain, ytrain, Xtest,ytest)
from sklearn.metrics import classification_report
print(classification_report(ytest, predict_label))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

CPU times: user 6.45 s, sys: 2.06 s, total: 8.51 s
Wall time: 6.04 s
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.13      0.36      0.19        11
           5       0.74      0.65      0.69       136
           6       0.70      0.59      0.64       128
           7       0.53      0.60      0.56        40
           8       0.25      0.67      0.36         3

    accuracy                           0.61       320
   macro avg       0.39      0.48      0.41       320
weighted avg       0.67      0.61      0.63       320



C1

In [ ]:
from sklearn.model_selection import train_test_split

#Xtrain, Xtest, ytrain, ytest = train_test_split(feature, label,test_size=0.2,random_state=22, stratify=label)
c1, c2, c3, c4, c5 = [], [], [], [], []
feature = data[['fixed acidity',	'volatile acidity',	'citric acid', 'residual sugar',	'chlorides',	'free sulfur dioxide',
                    	'total sulfur dioxide',	'density',	'pH',	'sulphates',	'alcohol']]
label = data['quality']


C1_feature, Xtest, ytrain, ytest = train_test_split(feature, label,test_size=0.2,random_state=22, stratify=label)

C1_feature = C1_feature.sort_index()
ytrain = ytrain.sort_index()
Xtest = Xtest.sort_index()
ytest = ytest.sort_index()
for item in ytrain:
  if item == 3 or item == 4 or item == 5:
    c1.append(1)
  else:
    c1.append(-1)


C1_test_label = RandomForest_2(C1_feature, c1, Xtest,ytest)
Xtest.insert(0,'C1',C1_test_label, allow_duplicates=False)
#Xtest




In [ ]:
Xtest

,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
4,1,8.3,1.020,0.02,3.4,0.084,6.0,11.0,0.99892,3.48,0.49,11.0
5,1,7.6,1.580,0.00,2.1,0.137,5.0,9.0,0.99476,3.50,0.40,10.9
18,1,7.6,0.680,0.02,1.3,0.072,9.0,20.0,0.99650,3.17,1.08,9.2
24,1,8.1,0.870,0.00,3.3,0.096,26.0,61.0,1.00025,3.60,0.72,9.8
28,1,8.2,0.915,0.27,2.1,0.088,7.0,23.0,0.99620,3.26,0.47,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1562,-1,7.7,0.280,0.30,2.0,0.062,18.0,34.0,0.99520,3.28,0.90,11.3
1575,-1,6.4,0.310,0.09,1.4,0.066,15.0,28.0,0.99459,3.42,0.70,10.0
1588,-1,10.7,0.350,0.53,2.6,0.070,5.0,16.0,0.99720,3.15,0.65,11.0
1589,-1,5.0,0.420,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0


In [ ]:
import collections
print(collections.Counter(Xtest.reset_index()[0:149].get('C1')))
print(collections.Counter(Xtest.reset_index()[149:320].get('C1')))

Counter({1: 123, -1: 26})
Counter({-1: 129, 1: 42})


C2

In [ ]:
c2 = []
for item in ytrain:
  if item == 3 or item == 4:
    c2.append(1)
  elif item == 5:
    c2.append(-1)
c2
C2_C3_data = C1_feature
C2_C3_data.insert(0,'C1',c1, allow_duplicates=False)
C2_C3_num = collections.Counter(C2_C3_data.get('C1'))

C2_train_data = C2_C3_data[0:C2_C3_num.get(1)]
C3_train_data = C2_C3_data[C2_C3_num.get(1):C2_C3_num.get(1)+C2_C3_num.get(-1)]
C2_test_data = Xtest[(Xtest.C1==1)]
C3_test_data = Xtest[(Xtest.C1==-1)]
C2_test_label = RandomForest_2(C2_train_data, c2, C2_test_data,ytest)


In [ ]:
C2_test_data.insert(0,'C2',C2_test_label, allow_duplicates=False)
predict_level_5 = C2_test_data[(C2_test_data.C1==1) & (C2_test_data.C2==-1)]
predict_level_5

,C2,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
4,-1,1,8.3,1.020,0.02,3.4,0.084,6.0,11.0,0.99892,3.48,0.49,11.0
5,-1,1,7.6,1.580,0.00,2.1,0.137,5.0,9.0,0.99476,3.50,0.40,10.9
18,-1,1,7.6,0.680,0.02,1.3,0.072,9.0,20.0,0.99650,3.17,1.08,9.2
24,-1,1,8.1,0.870,0.00,3.3,0.096,26.0,61.0,1.00025,3.60,0.72,9.8
28,-1,1,8.2,0.915,0.27,2.1,0.088,7.0,23.0,0.99620,3.26,0.47,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,-1,1,8.4,0.390,0.10,1.7,0.075,6.0,25.0,0.99581,3.09,0.43,9.7
1343,-1,1,6.8,0.470,0.08,2.2,0.064,18.0,38.0,0.99553,3.30,0.65,9.6
1354,-1,1,11.1,0.440,0.42,2.2,0.064,14.0,19.0,0.99758,3.25,0.57,10.4
1365,-1,1,5.6,0.310,0.78,13.9,0.074,23.0,92.0,0.99677,3.39,0.48,10.5


C3

In [ ]:
c3 = []
for item in ytrain:
  if item == 7 or item == 8:
    c3.append(1)
  elif item == 6:
    c3.append(-1)
C3_test_label = RandomForest_2(C3_train_data, c3, C3_test_data,ytest)
C3_test_data.insert(0,'C3',C3_test_label, allow_duplicates=False)
C3_test_data

,C3,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
34,-1,-1,7.5,1.115,0.10,3.1,0.086,5.0,12.0,0.99580,3.54,0.60,11.2
39,-1,-1,7.1,0.470,0.00,2.2,0.067,7.0,14.0,0.99517,3.40,0.58,10.9
46,-1,-1,6.0,0.330,0.32,12.9,0.054,6.0,113.0,0.99572,3.30,0.56,11.5
49,-1,-1,6.3,1.020,0.00,2.0,0.083,17.0,24.0,0.99437,3.59,0.55,11.2
51,1,-1,8.5,0.400,0.40,6.3,0.050,3.0,10.0,0.99566,3.28,0.56,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,1,-1,7.7,0.280,0.30,2.0,0.062,18.0,34.0,0.99520,3.28,0.90,11.3
1575,-1,-1,6.4,0.310,0.09,1.4,0.066,15.0,28.0,0.99459,3.42,0.70,10.0
1588,1,-1,10.7,0.350,0.53,2.6,0.070,5.0,16.0,0.99720,3.15,0.65,11.0
1589,1,-1,5.0,0.420,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0


In [ ]:
predict_level_6 = C3_test_data[(C3_test_data.C1==-1) & (C3_test_data.C3==-1)]
predict_level_6

,C3,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
34,-1,-1,7.5,1.115,0.10,3.1,0.086,5.0,12.0,0.99580,3.54,0.60,11.2
39,-1,-1,7.1,0.470,0.00,2.2,0.067,7.0,14.0,0.99517,3.40,0.58,10.9
46,-1,-1,6.0,0.330,0.32,12.9,0.054,6.0,113.0,0.99572,3.30,0.56,11.5
49,-1,-1,6.3,1.020,0.00,2.0,0.083,17.0,24.0,0.99437,3.59,0.55,11.2
61,-1,-1,6.8,0.910,0.06,2.0,0.060,4.0,11.0,0.99592,3.53,0.64,10.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533,-1,-1,11.6,0.410,0.54,1.5,0.095,22.0,41.0,0.99735,3.02,0.76,9.9
1534,-1,-1,11.6,0.410,0.54,1.5,0.095,22.0,41.0,0.99735,3.02,0.76,9.9
1547,-1,-1,10.6,0.360,0.57,2.3,0.087,6.0,20.0,0.99676,3.14,0.72,11.1
1575,-1,-1,6.4,0.310,0.09,1.4,0.066,15.0,28.0,0.99459,3.42,0.70,10.0


C4

In [ ]:
C2_train_data.insert(0,'C2',c2,allow_duplicates=False)
C2_train_data

,C2,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1,1,11.6,0.580,0.66,2.20,0.074,10.0,47.0,1.00080,3.25,0.57,9.0
1,1,1,10.4,0.610,0.49,2.10,0.200,5.0,16.0,0.99940,3.16,0.63,8.4
2,1,1,7.4,1.185,0.00,4.25,0.097,5.0,14.0,0.99660,3.63,0.54,10.7
3,1,1,10.4,0.440,0.42,1.50,0.145,34.0,48.0,0.99832,3.38,0.86,9.9
6,1,1,6.8,0.815,0.00,1.20,0.267,16.0,29.0,0.99471,3.32,0.51,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,-1,1,7.3,0.690,0.32,2.20,0.069,35.0,104.0,0.99632,3.33,0.51,9.5
737,-1,1,6.2,0.560,0.09,1.70,0.053,24.0,32.0,0.99402,3.54,0.60,11.3
739,-1,1,6.1,0.715,0.10,2.60,0.053,13.0,27.0,0.99362,3.57,0.50,11.9
740,-1,1,6.2,0.460,0.29,2.10,0.074,32.0,98.0,0.99578,3.33,0.62,9.8


In [ ]:
C4_test_data = C2_test_data[(C2_test_data.C1==1)&(C2_test_data.C2==1)]

In [ ]:
C4_train_data = C2_train_data[(C2_train_data.C1==1)&(C2_train_data.C2==1)]
c4 = []
for item in ytrain:
  if item == 3:
    c4.append(1)
  elif item == 4:
    c4.append(-1)
if len(C4_test_data) >0:
  C4_test_label = RandomForest_2(C4_train_data,c4,C4_test_data,ytest)
  C4_test_data.insert(0,'C4',C4_test_label, allow_duplicates=False)
else:
  C4_test_data.insert(0,'C4',0, allow_duplicates=False)
C4_test_data

,C4,C2,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
572,-1,1,1,6.6,0.8,0.03,7.8,0.079,6.0,12.0,0.9963,3.52,0.5,12.2
764,-1,1,1,4.7,0.6,0.17,2.3,0.058,17.0,106.0,0.9932,3.85,0.6,12.9


In [ ]:
predict_level_3 = C4_test_data[(C4_test_data.C1==1)&(C4_test_data.C2==1)&(C4_test_data.C4==1)]
predict_level_4 = C4_test_data[(C4_test_data.C1==1)&(C4_test_data.C2==1)&(C4_test_data.C4==-1)]
predict_level_4

,C4,C2,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
572,-1,1,1,6.6,0.8,0.03,7.8,0.079,6.0,12.0,0.9963,3.52,0.5,12.2
764,-1,1,1,4.7,0.6,0.17,2.3,0.058,17.0,106.0,0.9932,3.85,0.6,12.9


C5

In [ ]:
C3_train_data.insert(0,'C3',c3,allow_duplicates=False)
C3_train_data

,C3,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
744,-1,-1,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
745,-1,-1,7.9,0.320,0.51,1.8,0.341,17.0,56.0,0.99690,3.04,1.08,9.2
746,-1,-1,8.9,0.220,0.48,1.8,0.077,29.0,60.0,0.99680,3.39,0.53,9.4
747,-1,-1,6.9,0.400,0.14,2.4,0.085,21.0,40.0,0.99680,3.43,0.63,9.7
748,-1,-1,7.8,0.645,0.00,2.0,0.082,8.0,16.0,0.99640,3.38,0.59,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,1,-1,7.9,0.540,0.34,2.5,0.076,8.0,17.0,0.99235,3.20,0.72,13.1
1594,1,-1,8.6,0.420,0.39,1.8,0.068,6.0,12.0,0.99516,3.35,0.69,11.7
1595,1,-1,5.5,0.490,0.03,1.8,0.044,28.0,87.0,0.99080,3.50,0.82,14.0
1597,1,-1,7.2,0.380,0.31,2.0,0.056,15.0,29.0,0.99472,3.23,0.76,11.3


In [ ]:
C5_test_data = C3_test_data[(C3_test_data.C1==-1)&(C3_test_data.C3==1)]
C5_test_data

,C3,C1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
51,1,-1,8.5,0.400,0.40,6.30,0.050,3.0,10.0,0.99566,3.28,0.56,12.0
859,1,-1,13.8,0.490,0.67,3.00,0.093,6.0,15.0,0.99860,3.02,0.93,12.0
1074,1,-1,9.3,0.380,0.48,3.80,0.132,3.0,11.0,0.99577,3.23,0.57,13.2
1078,1,-1,8.6,0.315,0.40,2.20,0.079,3.0,6.0,0.99512,3.27,0.67,11.9
1087,1,-1,8.8,0.300,0.38,2.30,0.060,19.0,72.0,0.99543,3.39,0.72,11.8
1088,1,-1,8.8,0.300,0.38,2.30,0.060,19.0,72.0,0.99543,3.39,0.72,11.8
1139,1,-1,9.9,0.320,0.56,2.00,0.073,3.0,8.0,0.99534,3.15,0.73,11.4
1215,1,-1,8.2,0.340,0.37,1.90,0.057,43.0,74.0,0.99408,3.23,0.81,12.0
1221,1,-1,10.5,0.360,0.47,2.20,0.074,9.0,23.0,0.99638,3.23,0.76,12.0
1289,1,-1,7.5,0.580,0.56,3.10,0.153,5.0,14.0,0.99476,3.21,1.03,11.6


In [ ]:
C5_train_data = C3_train_data[(C3_train_data.C1==-1)&(C3_train_data.C3==1)]
c5 = []
for item in ytrain:
  if item == 7:
    c5.append(1)
  elif item == 8:
    c5.append(-1)
C5_test_label = RandomForest_2(C5_train_data, c5, C5_test_data,ytest)
C5_test_data.insert(0,'C5',C5_test_label, allow_duplicates=False)


In [ ]:
C5_test_data
predict_level_7 = C5_test_data[(C5_test_data.C1==-1)&(C5_test_data.C3==1)&(C5_test_data.C5==1)]
predict_level_8 = C5_test_data[(C5_test_data.C1==-1)&(C5_test_data.C3==1)&(C5_test_data.C5==-1)]
predict_level_7.insert(0,'predict_quality',7,allow_duplicates=False)
predict_level_3.insert(0,'predict_quality',3,allow_duplicates=False)
predict_level_4.insert(0,'predict_quality',4,allow_duplicates=False)
predict_level_5.insert(0,'predict_quality',5,allow_duplicates=False)
predict_level_6.insert(0,'predict_quality',6,allow_duplicates=False)
predict_level_8.insert(0,'predict_quality',8,allow_duplicates=False)
predict_level_3 = predict_level_3.drop(columns=['C1','C2','C4'])
predict_level_4 = predict_level_4.drop(columns=['C1','C2','C4'])
predict_level_5 = predict_level_5.drop(columns=['C1','C2'])
predict_level_6 = predict_level_6.drop(columns=['C1','C3'])
predict_level_8 = predict_level_8.drop(columns=['C1','C3','C5'])
predict_level_7 = predict_level_7.drop(columns=['C1','C3','C5'])
pred_df = pd.concat([predict_level_3,predict_level_4], axis=0)
pred_df = pd.concat([pred_df,predict_level_5], axis=0)
pred_df = pd.concat([pred_df,predict_level_6], axis=0)
pred_df = pd.concat([pred_df,predict_level_7], axis=0)
pred_df = pd.concat([pred_df,predict_level_8], axis=0)
pred_df = pred_df.sort_index()
pred_df

,predict_quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
4,5,8.3,1.020,0.02,3.4,0.084,6.0,11.0,0.99892,3.48,0.49,11.0
5,5,7.6,1.580,0.00,2.1,0.137,5.0,9.0,0.99476,3.50,0.40,10.9
18,5,7.6,0.680,0.02,1.3,0.072,9.0,20.0,0.99650,3.17,1.08,9.2
24,5,8.1,0.870,0.00,3.3,0.096,26.0,61.0,1.00025,3.60,0.72,9.8
28,5,8.2,0.915,0.27,2.1,0.088,7.0,23.0,0.99620,3.26,0.47,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1562,7,7.7,0.280,0.30,2.0,0.062,18.0,34.0,0.99520,3.28,0.90,11.3
1575,6,6.4,0.310,0.09,1.4,0.066,15.0,28.0,0.99459,3.42,0.70,10.0
1588,8,10.7,0.350,0.53,2.6,0.070,5.0,16.0,0.99720,3.15,0.65,11.0
1589,7,5.0,0.420,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, pred_df.predict_quality))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        11
           5       0.71      0.85      0.77       136
           6       0.61      0.59      0.60       128
           7       0.56      0.45      0.50        40
           8       1.00      0.33      0.50         3

    accuracy                           0.65       320
   macro avg       0.48      0.37      0.39       320
weighted avg       0.63      0.65      0.63       320



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
